In [1]:
import os
from tqdm import tqdm
from PIL import Image
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from drive.MyDrive.ClothSegmentation.data.base_dataset import Normalize_image
from drive.MyDrive.ClothSegmentation.utils.saving_utils import load_checkpoint_mgpu
from drive.MyDrive.ClothSegmentation.networks import U2NET

In [4]:
device = 'cuda'

image_dir = '/content/drive/My Drive/ClothSegmentation/input_images'
result_dir = '/content/drive/My Drive/ClothSegmentation/output_images'
checkpoint_path = '/content/drive/My Drive/ClothSegmentation/trained_checkpoint/cloth_segm.pth'

def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()

palette = get_palette(4)

!rm -rf input_images/.ipynb_checkpoints
images_list = sorted(os.listdir(image_dir))
pbar = tqdm(total=len(images_list))
for image_name in images_list:
    img = Image.open(os.path.join(image_dir, image_name)).convert('RGB')
    img_size = img.size
    img = img.resize((768, 768))
    image_tensor = transform_rgb(img)
    image_tensor = torch.unsqueeze(image_tensor, 0)

    output_tensor = net(image_tensor.to(device))
    output_tensor = F.log_softmax(output_tensor[0], dim=1)
    output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_arr = output_tensor.cpu().numpy()

    output_img = Image.fromarray(output_arr.astype('uint8'), mode='L')
    output_img = output_img.resize(img_size, Image.BICUBIC)

    output_img.putpalette(palette)
    output_img.save(os.path.join(result_dir, image_name[:-4]+'_generated.png'))

    plt.imshow(output_img)
    plt.title(image_name)

    pbar.update(1)

    pbar.update(1)

pbar.close()

----checkpoints loaded from path: /content/drive/My Drive/ClothSegmentation/trained_checkpoint/cloth_segm.pth----


RuntimeError: ignored

In [ ]:
import cv2
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow

original_image_path = '/content/drive/My Drive/ClothSegmentation/input_images/123.png'
segmented_image_path = '/content/drive/My Drive/ClothSegmentation/output_images/123_generated.png'

img = cv2.imread(original_image_path)

output_img = Image.open(segmented_image_path)
output_arr = np.array(output_img)

binary_mask = (output_arr > 0).astype(np.uint8) * 255

In [ ]:
# bounding box
# contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# for contour in contours:
#     x, y, w, h = cv2.boundingRect(contour)
#     cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# cv2_imshow(img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# get frame()
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    cv2.drawContours(img, [contour], -1, (0, 255, 0), 5)

cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.